# K-Nearest Neighbours

Willkommen, in diesem Notebook lernst du, wie du einen eigenen Klassifikator implementieren kannst, der mit scikit learn kompatible ist. Als grundlage werden wir den K-Nearest Neighbour Algorithmus implementieren. Es gibt bereits eine sehr gute Implementierung durch die `KNeighboursClassifiert` Klasse aus sklearn, daher ist es nicht ratsam für "echte" projekte eine eigene implementierung zu verwenden. In diesem Notebook soll es darum gehen, den Aufbau eines Klassifizierers in sklearn und den KNN algorithmus besser zu verstehen.

## Distanz- und Ähnlichkeitsfunktionen

Der KNN-Algorithmus ist ein überwachtes Machine-Learning-Verfahren, das sowohl für die Klassifikation als auch für die Regression verwendet werden kann. In diesem Notebook konzentrieren wir uns auf seine Anwendung als Klassifikator.

Die Grundidee des Verfahrens besteht darin, einem Datenpunkt die Klasse zuzuweisen, die in unmittelbarer Nähe des Punktes am häufigsten vorkommt. Zunächst müssen wir klären, was mit **Umgebung** gemeint ist.

Wie bei den meisten Machine-Learning-Verfahren müssen wir unsere Datenpunkte als Vektoren in einem n-dimensionalen Vektorraum darstellen können. Das bedeutet im Grunde genommen, dass ein Datenpunkt $X$ eine Sequenz von Zahlen $x_1, x_2, ..., x_n$ darstellt. Die Länge $n$ der Sequenz $X$ ist dabei immer fest.

Um die **Umgebung** eines Datenpunkts im KNN-Algorithmus zu definieren, benötigen wir geeignete Distanz- oder Ähnlichkeitsmetriken. Diese Metriken helfen uns, die Nähe zwischen Datenpunkten zu quantifizieren. Im Folgenden betrachten wir drei gängige Metriken: die euklidische Distanz, die Manhattan-Distanz und die Cosinus-Ähnlichkeit.

### Euklidische Distanz

Die euklidische Distanz ist wahrscheinlich die am häufigsten verwendete Metrik und berechnet die "Luftlinie" zwischen zwei Punkten in einem n-dimensionalen Raum. Sie wird durch die folgende Formel beschrieben:

$$
d_{euclid}(X, Y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

Hierbei sind $X$ und $Y$ zwei Datenpunkte, dargestellt als Vektoren $(x_1, x_2, ..., x_n)$ und $(y_1, y_2, ..., y_n)$. Die euklidische Distanz misst die direkte Strecke zwischen den Punkten und ist besonders nützlich, wenn die Datenpunkte kontinuierliche Werte haben.

![Euklidische Distanz](images/euclidean_distance.jpg)

### Manhattan-Distanz

Die Manhattan-Distanz, auch bekannt als L1-Distanz oder Taxicab-Distanz, summiert die absoluten Differenzen der einzelnen Komponenten. Sie wird durch die folgende Formel beschrieben:

$$
d_{manhattan}(X, Y) = \sum_{i=1}^{n} |x_i - y_i|
$$

Diese Metrik misst die Strecke entlang der Achsen des Vektorraums, ähnlich wie ein Taxi in einer Stadt mit einem rechtwinkligen Straßennetz fahren würde. Die Manhattan-Distanz kann besonders nützlich sein, wenn die Daten viele Ausreißer enthalten oder diskrete Werte angenommen werden.

![Manhattan-Distanz](images/manhattan_distance.jpg)

### Cosinus-Ähnlichkeit

Die Cosinus-Ähnlichkeit unterscheidet sich von den vorherigen Metriken, da sie den Winkel zwischen zwei Vektoren im n-dimensionalen Raum misst, anstatt die tatsächliche Distanz. Sie wird durch die folgende Formel beschrieben:

$$
\text{cos}(X, Y) = \frac{X \cdot Y}{\|X\| \|Y\|} = \frac{\sum_{i=1}^{n} x_i y_i}{\sqrt{\sum_{i=1}^{n} x_i^2} \sqrt{\sum_{i=1}^{n} y_i^2}}
$$

Hierbei ist $X \cdot Y$ das Skalarprodukt der Vektoren $X$ und $Y$, und $\|X\|$ sowie $\|Y\|$ sind die euklidischen Normen der Vektoren $X$ und $Y$.

Die Cosinus-Ähnlichkeit nimmt Werte zwischen -1 und 1 an, wobei 1 bedeutet, dass die Vektoren in die gleiche Richtung zeigen, 0 bedeutet, dass sie orthogonal sind, und -1 bedeutet, dass sie in entgegengesetzte Richtungen zeigen. Diese Metrik ist besonders nützlich, wenn es darum geht, die Richtung von Vektoren zu vergleichen, unabhängig von ihrer Größe.

![Cosinus-Ähnlichkeit](images/cosine_similarity.jpg)

### Anwendung im KNN-Algorithmus

Im KNN-Algorithmus bestimmen wir die "nächsten Nachbarn" eines Datenpunkts basierend auf diesen Distanz- oder Ähnlichkeitsmetriken. Die Wahl der Metrik kann die Leistung des Algorithmus erheblich beeinflussen und hängt oft von den spezifischen Eigenschaften der Daten ab.

Für die Implementierung in diesem Notebook werden wir uns zunächst auf die euklidische Distanz konzentrieren, da sie eine intuitive und weit verbreitete Methode ist, um die Nähe zwischen Datenpunkten zu messen. Sie ermöglicht klare und direkte Vergleiche zwischen den Punkten, was besonders hilfreich ist, um die Funktionsweise des KNN-Algorithmus zu verstehen.

In [1]:
import numpy as np
import pandas as pd

Beginne damit eine Funktion zur berechnung der Euklidischen Distanz zu schreiben. Die Funktion bekommt als eingabe zwei n-dimensionale Punkte in Form von numpy arrays und soll den Abstand nach Euclid als Zahl zurückgebe.
Hierfür sind die numpy Funktionen `np.sqrt` und `np.sum` zu verwenden.

In [2]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

In der Folgenden Zelle sind einige Testfälle mit denen du die Korrektheit deiner Implementierung überprüfen kannst.

In [3]:
# Testfälle

# Test 1: Identische Punkte
a = np.array([1, 2])
b = np.array([1, 2])
assert euclidean_distance(a, b) == 0.0, "Fehler in Test 1: Identische Punkte"

# Test 2: Einfache Punkte
a = np.array([0, 0])
b = np.array([3, 4])
assert euclidean_distance(a, b) == 5.0, "Fehler in Test 2: Einfache Punkte (3-4-5 Dreieck)"

# Test 3: Punkte mit negativen Werten
a = np.array([-1, -2])
b = np.array([1, 2])
assert np.isclose(euclidean_distance(a, b), 4.4721, atol=1e-4), "Fehler in Test 3: Punkte mit negativen Werten"

# Test 4: Höherdimensionale Punkte
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
assert np.isclose(euclidean_distance(a, b), 5.1962, atol=1e-4), "Fehler in Test 4: Höherdimensionale Punkte"

# Test 5: Höherdimensionale Punkte mit negativen Werten
# Test 5: Höherdimensionale Punkte mit negativen Werten
a = np.array([-1, -2, -3])
b = np.array([4, 5, 6])
expected_distance = np.sqrt(155)
assert np.isclose(euclidean_distance(a, b), expected_distance, atol=1e-4), "Fehler in Test 5: Höherdimensionale Punkte mit negativen Werten"

print("Alle Tests erfolgreich bestanden!")



Alle Tests erfolgreich bestanden!


Sehr gut. Nun wollen wir einen Schritt weiter gehen und auf basis unserer Abstandsmetrik die k-nächsten Nachbarn eines bestimmten Punktes aus einer Menge weiterer Punkte ermitteln.
Verfollständige die funktion `find_k_neighbours`. Die Funktion bekommt einen Punkt `point` sowie eine Sequenz von weiteren Punkten `X` sowie den Parameter `k`.
Zunächst soll der Abstand von `point` zu jedem weiteren Punkt in `X` ermittelt werden. Auf basis dieser Distanzen können dann die inidzes derjedingen Punkte gefunden werden, welche dem Punk `point` am nächsten liegen.

Für diese Aufgabe kannst du die zuvor geschriebene `euclidean_distance` Funktion verwenden. Zudem hilft dir die Funktion `np.argsort` die indizes nach der distanz zu sortieren.

Die Funktion `check_array` aus `sklearn.utils` ist Hilfreich um sicherzugehen, dass deine Funktion sowohl mit numpy arrays, als auch mit Pandas datenstrukturen funktioniert.

In [4]:
from sklearn.utils import check_array

def find_k_neighbours(point, X, k=5):
    """
    Findet die k nächsten Nachbarn eines gegebenen Punktes in einem Datensatz.

    Parameter:
    point (array-like): Ein eindimensionaler Vektor, der den Punkt darstellt, für den die nächsten Nachbarn gefunden werden sollen.
    X (array-like): Ein zweidimensionaler Datensatz, in dem die nächsten Nachbarn gesucht werden sollen. Jede Zeile repräsentiert einen Punkt im Raum.
    k (int, optional): Die Anzahl der nächsten Nachbarn, die gefunden werden sollen. Standardmäßig ist k=5.

    Rückgabe:
    indices (numpy.ndarray): Ein Array mit den Indizes der k nächsten Nachbarn im Datensatz X.

    """
    
    X = check_array(X)
    point = check_array(point, ensure_2d=False)
    
    # Berechnen Sie die euklidische Distanz zu jedem Punkt im Trainingssatz
    distances = [euclidean_distance(point, x) for x in X]

    # Finden Sie die Indizes der k nächsten Nachbarn
    indices = np.argsort(distances)[:k]

    return indices


In [5]:
# Testfall 1: Einfacher Testfall mit zwei Punkten
X = np.array([[1, 2], [3, 4]])
point = np.array([2, 3])
expected_result = np.array([0, 1])
assert np.array_equal(find_k_neighbours(point, X, k=2), expected_result)

# Testfall 2: Testfall mit drei Punkten, wobei einer der Punkte der gleiche wie der gegebene Punkt ist
X = np.array([[1, 2], [2, 3], [3, 4]])
point = np.array([2, 3])
expected_result = np.array([1, 0, 2])
assert np.array_equal(find_k_neighbours(point, X, k=3), expected_result)

# Testfall 3: Testfall mit mehreren Punkten und k=1
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
point = np.array([3, 4])
expected_result = np.array([2])
assert np.array_equal(find_k_neighbours(point, X, k=1), expected_result)

# Testfall 4: Testfall mit mehreren Punkten in höheren Dimensionen
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
point = np.array([5, 6, 7])
expected_result = np.array([1, 2, 0])
assert np.array_equal(find_k_neighbours(point, X, k=3), expected_result)

# Testfall 5: Testfall mit einem Pandas DataFrame
X = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [2, 3, 4, 5, 6],
    'z': [3, 4, 5, 6, 7]
})
point = np.array([3, 4, 5])
expected_result = np.array([2, 1, 3, 0, 4])
assert np.array_equal(find_k_neighbours(point, X, k=5), expected_result)

# Testfall 6: Testfall mit negativen Koordinaten
X = np.array([[-1, -2], [-3, -4], [-5, -6]])
point = np.array([-4, -5])
expected_result = np.array([1, 2, 0])
assert np.array_equal(find_k_neighbours(point, X, k=3), expected_result)

print("Alle Tests erfolgreich bestanden!")

Alle Tests erfolgreich bestanden!


Sehr gut. Lass uns nun die Funktion an einem realen Datensatz testen. Hierfür nehmen wir uns den Iris Datensatz zu Hilfe, welcher 150 Datenpunkte mit 4 Features und einer Targetklasse aus 3 möglichen Ausprägungen beinnhaltet.

In [6]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
X, y = iris.data, iris.target
display(X)
display(y)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int32

Nehmen wir beispielsweise den Punkt mit dem Index 45. Wenn wir uns für k=1 entscheiden und eine Punkt wählen der bereits in X enthalten ist, dann ist klar, dass genau dieser Punkt der nächstgelegene Nachbar ist.

In [7]:
find_k_neighbours(X.iloc[45, :], X, k=1)

array([45], dtype=int64)

Erweitern wir den Suchraum auf k=10 bekommen wir die Indizes der 10 nächstgelegenen Punkte. Diese Indizes können wir dann benutzen um die Target werte dieser Punkte aus y zu bestimmen:

In [8]:
point_idx = 56
k=10
neighbours_indizes = find_k_neighbours(X.iloc[point_idx, :], X, k=k)
neighbours_classes = y[neighbours_indizes]

print(f"{'index':<10} | {'class':<5}")
for neighbour_idx, neighbour_class in zip(neighbours_indizes, neighbours_classes):
    # print(neighbour_idx, neighbour_class)
    print(f"{neighbour_idx:>10} | {neighbour_class:<5}")


index      | class
        56 | 1    
        51 | 1    
        85 | 1    
        91 | 1    
       127 | 2    
        86 | 1    
        70 | 1    
       138 | 2    
        63 | 1    
        78 | 1    



Wir sehen, dass der Punkt mit dem Index 56 zur Klasse 1 gehört und dass sich in seiner unmittelbaren Nachbarschaft überwiegend Punkte aus eben dieser Klasse sowie ein paar Punkte der Klasse 2 befinden. Wir wollen nun die häufigste dieser Klassen auswählen.

Dafür können wir die Funktion `np.bincount` nutzen. Diese zählt die Häufigkeit jedes Wertes eines Arrays. Die Häufigkeiten werden dem Index des Arrays zugeordnet, dessen Wert sie darstellen. Das heißt, wenn der Wert 3 fünfmal vorkommt, gibt `bincount` ein Array zurück, an dessen dritter Stelle eine Fünf steht.


In [9]:
np.bincount([0, 0, 0, 1, 1, 3,])

array([3, 2, 0, 1], dtype=int64)

Der Wert 0 kommt drei mal vor, daher steht in der Ausgabe an der index position 0 der wert 3.

Zähle nun mit bincount die häufigkeit der zuvor ermittelten Klassen und speichere sie als variable `counts`.

In [10]:
# Zählen Sie die Anzahl der Instanzen jeder Klasse
counts = np.bincount(neighbours_classes) 
counts

array([0, 8, 2], dtype=int64)

Mit der Funktion `np.argmax` kannst du nun den Index mit der Größten Zahl ermitteln. Dieser Entspricht dann der Klasse, die am Häufigsten in der Nachbarschaft anzutreffen ist.

In [11]:
np.argmax(counts)

1

## Implementierung als Klasse

Sehr gut. Wir haben nun alle wichtigen Schritte besprochen, um den K-Nearest Neighbour Klassifizierer zu verstehen. Als nächstes wollen wir eine Klasse schreiben, um den Algorithmus wie einen `sklearn`-Classifier nutzen zu können.

### Grundaufbau

Dafür müssen wir die Grundstruktur von `sklearn`-Classifiern übernehmen. Ein Classifier in `sklearn` besitzt mindestens 3 Methoden:

- `__init__`: Diese Methode wird verwendet, um ein Objekt der Klasse zu instanzieren. Ihr wird der Parameter `k` übergeben, welcher als Attribut des Objekts abgespeichert wird.

- `fit`: Diese Methode wird für das Trainieren eines Classifiers verwendet. Sie dient dazu, unser Objekt mit den Trainingsdaten vertraut zu machen. Diese Trainingsdaten bilden letztlich die Nachbarschaft, in der nach ähnlichen Punkten gesucht wird. Da KNN ein Offline-Verfahren ist, reicht es, wenn wir uns die Trainingsdaten zunächst als Attribute des Objekts speichern. Allerdings ist zu beachten, dass diese Attribute mit einem `_` am Ende des Variablennamens enden.

- `predict`: Jeder Klassifikator implementiert die `predict`-Methode, um einen neuen Datenpunkt oder eine Reihe von Punkten zu klassifizieren. Diese Methode darf erst aufgerufen werden, wenn die `fit`-Methode bereits aufgerufen wurde. Um dies zu überprüfen, können wir die `check_is_fitted`-Funktion aus `sklearn.utils` verwenden. Diese überprüft, ob das Objekt Attribute mit einem `_` am Ende des Variablennamens besitzt. Danach können wir die bereits gelernten Techniken anwenden, um zu jedem Punkt eine Klasse aus den nächstgelegenen Nachbarn zu ermitteln. Hierfür bietet es sich an, die Funktion `euclidean_distance` als private Methode zu implementieren. Das hat den Vorteil, dass wir die Klasse leicht um weitere Metriken erweitern können.

### Einbeziehung von Basisklassen

Neben den von uns diskutierten Methoden `__init__`, `fit` und `predict` ist es auch wichtig, dass wir unsere K-Nearest Neighbour-Klasse von den Basisklassen `BaseEstimator` und `ClassifierMixin` aus dem `sklearn.base`-Modul ableiten.

Die Klasse `BaseEstimator` gibt unseren eigenen Estimator-Klassen nützliche Methoden wie `get_params` und `set_params`, die uns erlauben, die Parameter unserer Klassifizierer im `sklearn`-Stil zu verwalten. Dies ist besonders nützlich, wenn wir unser Modell mit verschiedenen Parametern optimieren möchten, da wir die Parameter einfach als Argumente an `set_params` weitergeben können. Dies macht es beispielsweise möglich die Parameter eines Modells durch `GridSearch` oder `RandomizedSearch` optimieren zu lassen.

Die Klasse `ClassifierMixin` ergänzt unsere Klasse mit der `score`-Methode, die ein einfaches Mittel zur Bewertung der Leistung des Klassifikators auf Testdaten bietet. Sie berechnet standardmäßig die Genauigkeit des Klassifikators, kann aber mit anderen Metriken überschrieben werden, wenn dies für unseren Anwendungsfall erforderlich ist.

Um von diesen Klassen zu erben, fügen wir einfach `BaseEstimator` und `ClassifierMixin` in die Klassendefinition ein.


Durch die Verwendung dieser Basisklassen stellen wir sicher, dass unser K-Nearest Neighbor-Klassifizierer nahtlos in die `sklearn`-Pipeline und verwandte Funktionen integriert werden kann.

Implementiere nun die Klasse mit den entsprechenden Funktionen.

In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin

class SimpleKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3):
        ...

    def _euclidean_distance(self, a, b):
        ...

    def fit(self, X, y):
        ...
        return self

    def predict(self, X):
        ...
        return np.array(predictions)


In [13]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import numpy as np

class SimpleKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3):
        self.k = k

    def _euclidean_distance(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2))

    def fit(self, X, y):
        # Überprüfen, dass X und y die richtige Form haben
        X, y = check_X_y(X, y)
        self.X_ = X
        self.y_ = y
        self.classes_, self.y_ = np.unique(y, return_inverse=True)
        return self

    def predict(self, X):
        # Überprüfen, ob fit aufgerufen wurde
        check_is_fitted(self)

        # Eingabe validieren
        X = check_array(X)

        # Vorhersagen für jeden Punkt
        predictions = []
        for x in X:
            # Berechnen Sie die euklidische Distanz zu jedem Punkt im Trainingssatz
            distances = [self._euclidean_distance(x, x_train) for x_train in self.X_]

            # Finden Sie die Indizes der k nächsten Nachbarn
            indices = np.argsort(distances)[:self.k]

            # Zählen Sie die Anzahl der Instanzen jeder Klasse
            counts = np.bincount(self.y_[indices], minlength=len(self.classes_))

            # Die Klasse mit den meisten Instanzen ist die Vorhersage
            prediction = self.classes_[np.argmax(counts)]
            predictions.append(prediction)

        return np.array(predictions)


Um unsere Implementierung fachgerecht zu testen müssen wir unsere Datanbasis in ein test- und ein train-Set aufteilen.

In [15]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y.to_frame())

In [18]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(112, 4)
(112, 1)
(38, 4)
(38, 1)


Nun können wir unser Modell mit dem traingsdatensatz fitten und danach mit dem test datensatz die Performance unseres Modells testen:

In [20]:
from sklearn.metrics import classification_report

In [21]:
sknn = SimpleKNNClassifier(k=5)
sknn.fit(X=X_train, y=y_train)

print(classification_report(y_test, sknn.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.92      1.00      0.96        11
           2       1.00      0.94      0.97        16

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



c:\Users\Nils Schillmann\.conda\envs\stackfuel\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Herzlichen Glückwunsch! Du hast erfolgreich einen eigenen Klassifikator erstellt, der nahtlos mit dem Ökosystem von Scikit-learn kompatibel ist.

Falls du nun Interesse daran hast, deine Implementierung zu optimieren, gibt es einige Möglichkeiten:

- Du könntest die Klasse in eine `.py`-Datei auslagern. Auf diese Weise kannst du die Klasse in beliebigen Skripten und Notebooks laden.
- Eine weitere Möglichkeit wäre, die Implementierung um zusätzliche Metriken wie die Manhattan-Distanz oder die Kosinus-Ähnlichkeit zu erweitern.
- Du könntest auch die Leistung verbessern. Unsere aktuelle Implementierung ist relativ einfach und könnte bei großen Datenmengen recht lange dauern. Die Implementierung kann weiter optimiert werden, damit beim Vorhersagen nicht zwingend der gesamte Trainingssatz durchsucht werden muss.

In [22]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import numpy as np

class SimpleKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3, distance_func='euclidean'):
        self.k = k
        self.distance_func = distance_func

    def _euclidean_distance(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2))

    def _manhattan_distance(self, a, b):
        return np.sum(np.abs(a - b))

    def _cosine_similarity(self, a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    def fit(self, X, y):
        # Überprüfen, dass X und y die richtige Form haben
        X, y = check_X_y(X, y)
        self.X_ = X
        self.y_ = y
        self.classes_, self.y_ = np.unique(y, return_inverse=True)
        return self

    def predict(self, X):
        # Überprüfen, ob fit aufgerufen wurde
        check_is_fitted(self)

        # Eingabe validieren
        X = check_array(X)

        # Wählen Sie die Distanzfunktion
        if self.distance_func == 'euclidean':
            distance_func = self._euclidean_distance
        elif self.distance_func == 'manhattan':
            distance_func = self._manhattan_distance
        elif self.distance_func == 'cosine':
            distance_func = self._cosine_similarity
        else:
            raise ValueError("Unbekannte Distanzfunktion: {}".format(self.distance_func))

        # Vorhersagen für jeden Punkt
        predictions = []
        for x in X:
            # Berechnen Sie die Distanz zu jedem Punkt im Trainingssatz
            distances = [distance_func(x, x_train) for x_train in self.X_]

            # Finden Sie die Indizes der k nächsten Nachbarn
            indices = np.argsort(distances)[:self.k]

            # Zählen Sie die Anzahl der Instanzen jeder Klasse
            counts = np.bincount(self.y_[indices], minlength=len(self.classes_))

            # Die Klasse mit den meisten Instanzen ist die Vorhersage
            prediction = self.classes_[np.argmax(counts)]
            predictions.append(prediction)

        return np.array(predictions)
